In [1]:
from manim import *

from rc_lib import math_types as T
from rc_lib import style
from rc_lib.math_utils import geometry
from rc_lib.design_utils import plate, sketch
from rc_lib.view_utils import title_sequence

quality = "ql"

Manim Community v0.17.2

The reccomended way to create plates is by first drawing your holes and other relevant geometry, marking out the outer boundary using a second set of circles, and then connecting the boundary with tangent lines.

In [2]:
%%manim -v WARNING --disable_caching -$quality IntakePlateScene

class IntakePlateScene(Scene):
    def construct(self):
        inner_color = style.Color.GREEN
        boundary_color = style.Color.BLUE

        small_base = plate.PlateCircleFactory(0.15, 0.2)
        small_base.set_inner_color(inner_color).set_outer_color(boundary_color)
        medium_base = small_base.copy(0.4)

        front_hole = geometry.point_2d(-4, -3)
        middle_hole = geometry.point_2d(-1.5, 0.25)
        back_hole = geometry.point_2d(2.5, 1.5)
        back_offset = geometry.point_2d(0.8, 0.75)

        points = [
            medium_base.make(front_hole),
            medium_base.make(middle_hole),
            medium_base.make(back_hole),
            small_base.make(back_hole + back_offset),
            small_base.make(back_hole + geometry.point_2d(1, -0.2)),
            small_base.make((middle_hole + back_hole) / 2),
            small_base.make((front_hole + middle_hole) / 2),
        ]

        boundary_order = [1, 3, 4, 0]
        plate_group = plate.PlateGroup(points, boundary_order, boundary_color=boundary_color)
        title = title_sequence.TitleSequence(add_numbers=True)

        self.play(title.next("Draw plate holes", color=inner_color))
        self.play(plate_group.draw_inner_circles(lag_ratio=0.75))
        self.wait(style.Time.STANDARD)

        self.play(title.next("Add larger circles", color=boundary_color))
        self.play(plate_group.draw_outer_circles(lag_ratio=0.75))
        self.wait(style.Time.STANDARD)

        self.play(title.next("Connect boundary", color=boundary_color))
        self.play(plate_group.draw_boundary(), run_time=style.Time.SLOW * 3)
        self.wait(style.Time.STANDARD)

        # self.play(title.next("Trim", color=boundary_color))
        # self.play(plate_group.trim(), run_time=style.Time.SLOW * 3)

        self.wait(style.Time.END)

In the event a hole is too close to the edge of a plate, you may need to redraw the boundary.

In [ ]:
%%manim -v WARNING --disable_caching -$quality BoundaryRedrawScene

class BoundaryRedrawScene(Scene):
    def construct(self):
        left = plate.PlateCircle.make(1.75, 0.75).move_to(geometry.point_2d(-6, -2))
        right = plate.PlateCircle.make(1.75, 0.75).move_to(geometry.point_2d(6, -2))
        self.add(left)
        self.add(right)

        line = plate.PlateCircle.tangent_line(left, right)
        self.add(line)

        middle = plate.PlateCircle.make(1, 0.75).move_to(geometry.point_2d(0, -0.75))
        self.add(middle.inner_circle())

        title = title_sequence.TitleSequence(
            ["Add outer circle", "Redraw boundary"])

        self.play(title.next())
        self.play(middle.draw_outer_circle())

        self.play(title.next())
        self.play(Uncreate(line))
        self.wait(style.Time.FAST)
        self.play(Create(plate.PlateCircle.tangent_line(left, middle)))
        self.play(Create(plate.PlateCircle.tangent_line(middle, right)))

        self.wait(style.Time.END)

In [ ]:
%%manim -v WARNING -$quality LineConstraintScene

class LineConstraintScene(Scene):
    def construct(self):
        left = plate.PlateCircle.make(1.75, 0.75).move_to(geometry.point_2d(-6, -2))
        right = plate.PlateCircle.make(1.75, 0.75).move_to(geometry.point_2d(6, -2))
        self.add(left)
        self.add(right)
        tangent_points = plate.PlateCircle.tangent_points(left, right)

        title = title_sequence.TitleSequence(["Create line", "Add coincident constraints", "Add tangent constraints"])

        self.play(title.next())
        left_start_point = tangent_points[0] + geometry.point_2d(1.75, 0.75)
        right_start_point = tangent_points[1] + geometry.point_2d(-2, 0.5)

        line = sketch.SketchLine(left_start_point, right_start_point)
        self.play(line.create())

        self.play(title.next())
        left_coincident_point = left.center() + normalize(left_start_point - left.center()) * left.outer_radius()
        self.flash(line.start_point(), left)
        self.play(line.move_start(left_coincident_point))

        right_coincident_point = right.center() + normalize(right_start_point - right.center()) * right.outer_radius()
        self.flash(line.end_point(), right)
        self.play(line.move_end(right_coincident_point))

        self.play(title.next())
        self.flash(line.start_point(), left)
        angle = angle_between_vectors(left_coincident_point - left.center(), tangent_points[0] - left.center())
        self.play(line.move_start(tangent_points[0]), path_arc=angle, path_arg_centers=[left.center()])

        self.flash(line.end_point(), right)
        angle = -angle_between_vectors(right_coincident_point - right.center(), tangent_points[1] - right.center())
        self.play(line.move_end(tangent_points[1]), path_arc=angle, path_arg_centers=[right.center()])

        self.wait(style.Time.END)
    
    def flash(self, point, circle):
        self.play(Flash(point), run_time=style.Time.MEDIUM)
        self.play(Flash(circle, flash_radius=circle.outer_radius(), num_lines=40), run_time=style.Time.MEDIUM)
